In [1]:
import pandas as pd
import numpy as np

import requests
import json

from notebooks.config import *

In [2]:
def get_channel_stats(channel_id, api_key):
    url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&channelId={channel_id}&maxResults=50&order=date&key={api_key}'

    video_data = []

    while True:
        request = requests.get(url)
        json_stats = json.loads(request.text)

        if 'items' not in json_stats:
            break

        videos = json_stats['items']

        for video in videos:
            video_snippet = video['snippet']
            video_title = video_snippet['title']
            video_published_date = video_snippet['publishedAt']

            if 'videoId' in video['id']:
                video_id = video['id']['videoId']
            else:
                continue

            video_stats_request = f'https://www.googleapis.com/youtube/v3/videos?part=statistics&id={video_id}&key={api_key}'
            video_stats_response = requests.get(video_stats_request)
            video_stats_json = json.loads(video_stats_response.text)

            if 'items' not in video_stats_json:
                continue

            video_stats = video_stats_json['items'][0]['statistics']

            video_views = video_stats.get('viewCount', 0)
            video_likes = video_stats.get('likeCount', 0)

            video_tags_request = f'https://www.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={api_key}'
            video_tags_response = requests.get(video_tags_request)
            video_tags_json = json.loads(video_tags_response.text)

            if 'items' not in video_tags_json:
                continue

            video_tags = video_tags_json['items'][0]['snippet'].get('tags', [])

            video_data.append({
                'Published Date': video_published_date,
                'Title': video_title,
                'Views': video_views,
                'Likes': video_likes,
                'Tags': video_tags
            })

        if 'nextPageToken' in json_stats:
            next_page_token = json_stats['nextPageToken']
            url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&channelId={channel_id}&maxResults=50&order=date&pageToken={next_page_token}&key={api_key}'
        else:
            break

    return pd.DataFrame(video_data)


In [3]:
channel_stats_df = get_channel_stats(CHANNEL_ID, API_KEY)
channel_stats_df

,Published Date,Title,Views,Likes,Tags
0,2023-05-10T17:30:05Z,Leer LIBROS físicos vs Leer en iPad y Audiolibros,2125,178,"[libros digitales, booktube, libros, audio lib..."
1,2023-05-05T17:00:03Z,Apps para Mac que mejorarán tu productividad,4459,298,"[top 5 apps para mac, mejores apps para mac, t..."
2,2023-05-01T17:00:06Z,Notion en Español y consejos para principiantes,11489,590,"[como usar notion en español, notion, cómo pon..."
3,2023-04-13T17:03:43Z,Nintendo Switch: 10 Trucos que tienes que conocer,4722,346,"[nintendo switch, trucos nintendo switch, nint..."
4,2023-04-09T18:00:12Z,Mi teclado mecánico FAVORITO para Mac 🥇,3249,159,"[nuphy, nuphy keyboard, mechanical keyboard, t..."
...,...,...,...,...,...
244,2019-10-05T22:20:32Z,ANÁLISIS | REVIEW del iPhone11 Pro Max en Espa...,1050,114,"[iPhone11ProReviewEspañol, iPhone11Pro, iPhone..."
245,2019-09-26T10:00:09Z,Así es el Nuevo iPhone11 Pro Max. Increíble.,680,60,"[iphone 11, iphone 11 pro, iphone 11 pro max, ..."
246,2019-09-20T10:00:19Z,Vivir en Estados Unidos. Capítulo 1,1486,131,"[#vivirenestadosunidos, #vivirenusa, #vivirene..."
247,2019-09-10T10:45:23Z,Las 3 mejores Apps de Realidad Aumentada,882,52,"[tecnología, realidadaumentada, ar, vr, realid..."


In [5]:
# Save dataframe
channel_stats_df.to_csv('data/channel_stats.csv', index=False)